# Diseño de experimento

El diseño de experimentos en el entrenamiento de redes neuronales juega un papel crucial en la optimización del rendimiento del modelo. Este enfoque sistemático permite explorar y ajustar los hiperparámetros (como la tasa de aprendizaje, el número de capas y neuronas, y el tamaño del lote) de manera eficiente y efectiva. Al planificar y estructurar los experimentos, se pueden identificar las combinaciones óptimas de hiperparámetros que mejoran la precisión y la generalización del modelo, mientras se minimiza el tiempo y los recursos computacionales necesarios. Además, un buen diseño de experimentos reduce el impacto ecológico del entrenamiento de redes neuronales al disminuir el número de ejecuciones necesarias, optimizando así el consumo de energía y hardware.

## Una variable a la vez

El enfoque de diseño de experimentos de una variable a la vez (One Variable At a Time, OVAT) es una metodología simple en la que se varía un solo factor o variable experimental mientras se mantienen constantes todos los demás factores. Este enfoque permite observar cómo cambios en esa única variable afectan el resultado del experimento. 

Equipo:

-Elias Nieto Víctor David

-Pérez Lucio Kevyn Alejandro

-Rojas Alarcon Sergio Ulises

-Trejo Arriaga Rodrigo Gerardo

In [23]:
# Cargamos paquetes necesarios

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import numpy as np
import time
import pandas as pd
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms

### 1. Definición de variables y rango de valores:

Identificar los factores o variables experimentales clave que se desean estudiar. En el contexto de redes neuronales, estas variables pueden incluir la tasa de aprendizaje, el número de capas, el número de neuronas por capa, el tamaño del lote, entre otros.

Variables que tomaremos en cuenta:

- Tasa de aprendizaje (eta)
- Número de épocas (n_epocas)
- Tamaño de lote (batch_size)

Una vez definidas las variables independientes definimos un rango de valores posibles para cada variable.

> TODO: Define un rango de valores posibles para cada variable. Incluye el valor mínimo y el valor máximo. Se sugiere utilizar una lista de valores obtenida con una separación uniforme. Probar almenos 5 valores por variable.




In [24]:
# Tasa de aprendizaje
eta_values = [0.1, 0.01, 0.001, 0.0001, 0.00001]

# Número de épocas
n_epocas_values = [10, 15, 20, 25, 30]

# Tamaño de lote
batch_size_values = [16, 32, 64, 128, 256]

### 2. Configuración inicial:

Establecer una configuración inicial para la red neuronal con valores predeterminados para todos los hiperparámetros. 

> TODO: Define la configuración inicial. Se sugiere usar un diccionario para contener dicha configuración.


In [25]:
configuracion = {
    'epochs': 10,
    'batch_size': 64
}

### 3. Variación de una variable a la vez:

- Seleccionar la primera variable a estudiar (por ejemplo, la tasa de aprendizaje).
- Realizar una serie de experimentos donde se varía únicamente la tasa de aprendizaje, mientras se mantienen constantes todos los demás hiperparámetros.
- Registrar el rendimiento del modelo para cada valor de la tasa de aprendizaje.

> TODO: Modifica el código para que pueda aceptar la configuración deseada

In [26]:
# Definimos una transformación de los datos
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])

# Descargamos el conjunto de entrenamiento y cargamos mediante un dataLoader
trainset = datasets.FashionMNIST('F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Descargamos el conjunto de validación
validationset = datasets.FashionMNIST('F_MNIST_data/', download=True, train=False, transform=transform)
validationloader = torch.utils.data.DataLoader(validationset, batch_size=64, shuffle=True)


In [27]:
# Definición del modelo de red neuronal
class RedNeuronal(nn.Module):
    def __init__(self, input_size, output_size, hidden_layers, drop_p=0.5):
        super().__init__()
        
        # Agregamos la primera capa
        self.hidden_layers = nn.ModuleList([nn.Linear(input_size, hidden_layers[0])])
        
        # Agregamos cada una de las capas, zip empareja el número de entradas con las salidas
        layer_sizes = zip(hidden_layers[:-1], hidden_layers[1:])
        self.hidden_layers.extend([nn.Linear(h1, h2) for h1, h2 in layer_sizes])
        
        # Agregamos la capa de salida final de la red
        self.output = nn.Linear(hidden_layers[-1], output_size)
        
        # Incluimos drop-out en la red
        self.dropout = nn.Dropout(p=drop_p)
        
    def forward(self, x):
        ''' Pase hacia adelante en la red, el regreso son las probabilidades en el dominio log '''
        for linear in self.hidden_layers:
            x = F.relu(linear(x))
            x = self.dropout(x)
        
        x = self.output(x)
        
        return F.log_softmax(x, dim=1)

In [28]:
def validation(model, validationloader, criterion):
    test_loss = 0
    accuracy = 0
    for images, labels in validationloader:
        images.resize_(images.shape[0], 784)
        output = model.forward(images)
        test_loss += criterion(output, labels).item()
        ps = torch.exp(output)
        equality = (labels.data == ps.max(dim=1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()
    return test_loss, accuracy

In [29]:
results = []

for i, eta in enumerate(eta_values):
    print(f"Experimento {i+1}: Evaluando tasa de aprendizaje {eta}")
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=configuracion['batch_size'], shuffle=True)
    validationloader = torch.utils.data.DataLoader(validationset, batch_size=configuracion['batch_size'], shuffle=True)
    
    model = RedNeuronal(784, 10, [516, 256], drop_p=0.5)
    criterion = nn.NLLLoss()
    optimizer = optim.Adam(model.parameters(), lr=eta)
    epochs = configuracion['epochs']
    steps = 0
    running_loss = 0
    print_every = 40
    
    for e in range(epochs):
        model.train()
        for images, labels in trainloader:
            steps += 1
            images.resize_(images.size()[0], 784)
            optimizer.zero_grad()
            output = model.forward(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        model.eval()
        with torch.no_grad():
            test_loss, accuracy = validation(model, validationloader, criterion)
        print(f"Epoch: {e+1}/{epochs}.. ",
              f"Tasa de aprendizaje: {eta}.. ",
              f"Pérdida de entrenamiento: {running_loss/print_every:.3f}.. ",
              f"Pérdida de validación: {test_loss/len(validationloader):.3f}.. ",
              f"Exactitud de validación: {accuracy/len(validationloader):.3f}")
        results.append([eta, epochs, configuracion['batch_size'], running_loss/print_every, test_loss/len(validationloader), accuracy/len(validationloader)])
        running_loss = 0
        model.train()

df_results = pd.DataFrame(results, columns=['Learning Rate', 'Epochs', 'Batch Size', 'Training Loss', 'Validation Loss', 'Validation Accuracy'])

Experimento 1: Evaluando tasa de aprendizaje 0.1
Epoch: 1/10..  Tasa de aprendizaje: 0.1..  Pérdida de entrenamiento: 673.249..  Pérdida de validación: 2.818..  Exactitud de validación: 0.101
Epoch: 2/10..  Tasa de aprendizaje: 0.1..  Pérdida de entrenamiento: 580.527..  Pérdida de validación: 2.624..  Exactitud de validación: 0.100
Epoch: 3/10..  Tasa de aprendizaje: 0.1..  Pérdida de entrenamiento: 1127.660..  Pérdida de validación: 3.945..  Exactitud de validación: 0.100
Epoch: 4/10..  Tasa de aprendizaje: 0.1..  Pérdida de entrenamiento: 1409.957..  Pérdida de validación: 7.246..  Exactitud de validación: 0.100
Epoch: 5/10..  Tasa de aprendizaje: 0.1..  Pérdida de entrenamiento: 1452.069..  Pérdida de validación: 2.489..  Exactitud de validación: 0.100
Epoch: 6/10..  Tasa de aprendizaje: 0.1..  Pérdida de entrenamiento: 1405.175..  Pérdida de validación: 2.332..  Exactitud de validación: 0.100
Epoch: 7/10..  Tasa de aprendizaje: 0.1..  Pérdida de entrenamiento: 1543.589..  Pérdida 

### 4. Selección del mejor valor:

Analizar los resultados y seleccionar el valor de la tasa de aprendizaje que produce el mejor rendimiento del modelo.


In [36]:
# Si se ejecuta por primera vez, descomentar la linea siguiente
# df_results['Validation Accuracy'] = df_results['Validation Accuracy'].apply(lambda x: x.item())
last_epoch_results = df_results.groupby(['Learning Rate']).tail(1).reset_index(drop=True)
best_result = last_epoch_results.loc[last_epoch_results['Validation Accuracy'].idxmax()]

In [37]:
last_epoch_results

,Learning Rate,Epochs,Batch Size,Training Loss,Validation Loss,Validation Accuracy
0,0.10000,10,64,657.125797,2.307695,0.099622
1,0.01000,10,64,28.055921,0.876105,0.658838
2,0.00100,10,64,8.940564,0.357506,0.868332
3,0.00010,10,64,8.074979,0.355960,0.869825
4,0.00001,10,64,12.799471,0.499012,0.818073


In [38]:
print("\nMejor tasa de aprendizaje:")
print(best_result['Learning Rate'])
print("\nResultados asociados al mejor valor:")
print(best_result)


Mejor tasa de aprendizaje:
0.0001

Resultados asociados al mejor valor:
Learning Rate           0.000100
Epochs                 10.000000
Batch Size             64.000000
Training Loss           8.074979
Validation Loss         0.355960
Validation Accuracy     0.869825
Name: 3, dtype: float64


### 5. Repetición para otras variables:

Proceder con la siguiente variable (por ejemplo, el número de capas) y repetir el proceso: variar solo esta variable mientras se mantienen constantes todos los demás hiperparámetros, utilizando el mejor valor encontrado para la tasa de aprendizaje. Continuar este proceso para cada variable en la lista.

> TODO: Escribe una tabla con el resultado de cada experimento. Las columnas deben ser: ID, Configuración, Exactitud obtenida.

#### Fijamos el learning rate en 0.0001 y variamos el tamaño del batch

In [39]:
learning_rate = 0.0001
epochs = 10
results = []

for i, batch_size in enumerate(batch_size_values):
    print(f"Experimento {i+1}: Evaluando tamaño de lote {batch_size}")
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
    validationloader = torch.utils.data.DataLoader(validationset, batch_size=batch_size, shuffle=True)
    
    model = RedNeuronal(784, 10, [516, 256], drop_p=0.5)
    criterion = nn.NLLLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    steps = 0
    running_loss = 0
    print_every = 40
    
    for e in range(epochs):
        model.train()
        for images, labels in trainloader:
            steps += 1
            images.resize_(images.size()[0], 784)
            optimizer.zero_grad()
            output = model.forward(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        model.eval()
        with torch.no_grad():
            test_loss, accuracy = validation(model, validationloader, criterion)
        print(f"Epoch: {e+1}/{epochs}.. ",
              f"Tamaño de lote: {batch_size}.. ",
              f"Pérdida de entrenamiento: {running_loss/print_every:.3f}.. ",
              f"Pérdida de validación: {test_loss/len(validationloader):.3f}.. ",
              f"Exactitud de validación: {accuracy/len(validationloader):.3f}")
        results.append([learning_rate, epochs, batch_size, running_loss/print_every, test_loss/len(validationloader), accuracy/len(validationloader)])
        running_loss = 0
        model.train()

df_batch_results = pd.DataFrame(results, columns=['Learning Rate', 'Epochs', 'Batch Size', 'Training Loss', 'Validation Loss', 'Validation Accuracy'])


Experimento 1: Evaluando tamaño de lote 16
Epoch: 1/10..  Tamaño de lote: 16..  Pérdida de entrenamiento: 63.151..  Pérdida de validación: 0.463..  Exactitud de validación: 0.830
Epoch: 2/10..  Tamaño de lote: 16..  Pérdida de entrenamiento: 43.787..  Pérdida de validación: 0.417..  Exactitud de validación: 0.846
Epoch: 3/10..  Tamaño de lote: 16..  Pérdida de entrenamiento: 39.576..  Pérdida de validación: 0.402..  Exactitud de validación: 0.853
Epoch: 4/10..  Tamaño de lote: 16..  Pérdida de entrenamiento: 37.214..  Pérdida de validación: 0.382..  Exactitud de validación: 0.859
Epoch: 5/10..  Tamaño de lote: 16..  Pérdida de entrenamiento: 35.398..  Pérdida de validación: 0.370..  Exactitud de validación: 0.865
Epoch: 6/10..  Tamaño de lote: 16..  Pérdida de entrenamiento: 34.101..  Pérdida de validación: 0.363..  Exactitud de validación: 0.871
Epoch: 7/10..  Tamaño de lote: 16..  Pérdida de entrenamiento: 33.056..  Pérdida de validación: 0.354..  Exactitud de validación: 0.873
Epoch

In [41]:
df_last_epoch = df_results[df_results['Epochs'] == epochs]

In [43]:
best_result = df_last_epoch.loc[df_last_epoch['Validation Accuracy'].idxmax()]

In [46]:
best_result

Learning Rate           0.000100
Epochs                 10.000000
Batch Size             64.000000
Training Loss           8.074979
Validation Loss         0.355960
Validation Accuracy     0.869825
Name: 39, dtype: float64

In [48]:
print("\nMejor tasa de aprendizaje:")
print(best_result['Batch Size'])
print("\nResultados asociados al mejor valor:")
print(best_result)


Mejor tasa de aprendizaje:
64.0

Resultados asociados al mejor valor:
Learning Rate           0.000100
Epochs                 10.000000
Batch Size             64.000000
Training Loss           8.074979
Validation Loss         0.355960
Validation Accuracy     0.869825
Name: 39, dtype: float64


#### Fijamos el learning rate en 0.0001 y el tamaño del batch en 64 mientras variamos el numero de épocas

In [52]:
learning_rate = 0.0001
batch_size = 64

results = []

for i, epochs in enumerate(n_epocas_values):
    print(f"Experimento {i+1}: Evaluando número de épocas {epochs}")
    model = RedNeuronal(784, 10, [516, 256], drop_p=0.5)
    criterion = nn.NLLLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    print_every = 40
    
    for e in range(epochs):
        model.train()
        running_loss = 0
        for images, labels in trainloader:
            images.resize_(images.size()[0], 784)
            optimizer.zero_grad()
            output = model.forward(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        model.eval()
        with torch.no_grad():
            test_loss, accuracy = validation(model, validationloader, criterion)
        print(f"Epoch: {e+1}/{epochs}.. ",
              f"Número de épocas: {epochs}.. ",
              f"Pérdida de entrenamiento: {running_loss/len(trainloader):.3f}.. ",
              f"Pérdida de validación: {test_loss/len(validationloader):.3f}.. ",
              f"Exactitud de validación: {accuracy/len(validationloader):.3f}")
        results.append([learning_rate, epochs, running_loss/len(trainloader), test_loss/len(validationloader), accuracy/len(validationloader)])
        model.train()


df_results = pd.DataFrame(results, columns=['Learning Rate', 'Epochs', 'Training Loss', 'Validation Loss', 'Validation Accuracy'])

Experimento 1: Evaluando número de épocas 5
Epoch: 1/5..  Número de épocas: 5..  Pérdida de entrenamiento: 1.066..  Pérdida de validación: 0.612..  Exactitud de validación: 0.777
Epoch: 2/5..  Número de épocas: 5..  Pérdida de entrenamiento: 0.619..  Pérdida de validación: 0.524..  Exactitud de validación: 0.806
Epoch: 3/5..  Número de épocas: 5..  Pérdida de entrenamiento: 0.538..  Pérdida de validación: 0.473..  Exactitud de validación: 0.825
Epoch: 4/5..  Número de épocas: 5..  Pérdida de entrenamiento: 0.495..  Pérdida de validación: 0.458..  Exactitud de validación: 0.831
Epoch: 5/5..  Número de épocas: 5..  Pérdida de entrenamiento: 0.464..  Pérdida de validación: 0.443..  Exactitud de validación: 0.840
Experimento 2: Evaluando número de épocas 10
Epoch: 1/10..  Número de épocas: 10..  Pérdida de entrenamiento: 1.091..  Pérdida de validación: 0.619..  Exactitud de validación: 0.775
Epoch: 2/10..  Número de épocas: 10..  Pérdida de entrenamiento: 0.629..  Pérdida de validación: 0.

In [56]:
df_last_epoch = df_results[df_results.groupby('Epochs')['Epochs'].transform('max') == df_results['Epochs']]
best_result = df_last_epoch.loc[df_last_epoch['Validation Accuracy'].idxmax()]

print("\nMejor número de épocas:")
print(best_result['Epochs'])
print("\nResultados asociados al mejor valor:")
print(best_result)


Mejor número de épocas:
25

Resultados asociados al mejor valor:
Learning Rate                  0.0001
Epochs                             25
Training Loss                0.305473
Validation Loss              0.334719
Validation Accuracy    tensor(0.8825)
Name: 74, dtype: object


## Conclusiones

¿Cual fue el mejor valor encontrado?
¿Cuantas ejecuciones se realizaron?
¿Que tiempo tomó realizar todos los experimentos?

### Mejor valor encontrado:
El mejor valor encontrado en los experimentos fue con una tasa de aprendizaje de **0.0001**, un tamaño de lote de **64** y **25** épocas. Este conjunto de hiperparámetros resultó en la mejor exactitud de validación.

### Número de ejecuciones realizadas:
En total, se realizaron **15** ejecuciones experimentales:
- **5** ejecuciones variando la tasa de aprendizaje.
- **5** ejecuciones variando el tamaño del lote.
- **5** ejecuciones variando el número de épocas.

### Tiempo total de los experimentos:
El tiempo total para realizar todos los experimentos fue de aproximadamente **35 minutos y 50 segundos**, desglosado de la siguiente manera:
- Experimentos variando la tasa de aprendizaje: **11 minutos y 54 segundos**.
- Experimentos variando el tamaño del lote: **11 minutos y 13 segundos**.
- Experimentos variando el número de épocas: **12 minutos y 43 segundos**.

Estos resultados muestran que la red neuronal alcanza su mejor rendimiento con una configuración específica de hiperparámetros y que el tiempo de entrenamiento puede variar dependiendo de los valores de los hiperparámetros utilizados. La configuración óptima hallada puede ser utilizada como referencia para futuros modelos y tareas similares, optimizando así tanto el tiempo de entrenamiento como la precisión del modelo.


El diseño de experimentos es crucial en el entrenamiento de redes neuronales para identificar configuraciones óptimas de hiperparámetros. Utilizar la metodología de variar una variable a la vez (OVAT) ofrece varias ventajas significativas. En primer lugar, proporciona simplicidad y claridad al permitir observar el efecto directo de un solo hiperparámetro en el rendimiento del modelo, facilitando así la comprensión y el ajuste de los parámetros.

Además, OVAT reduce la complejidad computacional y es ideal para la optimización iterativa, permitiendo ajustes progresivos basados en los resultados obtenidos. Esto mejora la eficiencia de los programadores en proyectos de Deep Learning, al simplificar el proceso de optimización y permitir una mejor gestión de los recursos computacionales.
